In [75]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import *

def print_auc(click, display, pctr):
    data = zip(click, display, pctr)
    data = sorted(data, key=lambda x:x[2], reverse=True)
    last_ctr = data[0][2] + 1.0
    auc_temp = 0.0
    click_sum = 0.0
    old_click_sum = 0.0
    no_click = 0.0
    no_click_sum = 0.0
    for i in data:
        if last_ctr != i[2]:
            auc_temp += (click_sum + old_click_sum) * no_click / 2.0
            old_click_sum = click_sum
            no_click = 0.0
            last_ctr = i[2]
        no_click += i[1] - i[0]
        no_click_sum += i[1] - i[0]
        click_sum += i[0]
    auc_temp += (click_sum+old_click_sum) * no_click / 2.0
    auc = auc_temp / (click_sum * no_click_sum)
    print auc

def mask(x):
    bitarr = [0]*10
    bitarr[x] = 1
    return bitarr
    
def gen(filename):
    train = np.loadtxt(filename, delimiter=",", skiprows=0)
    X = train[:,:-1]
    y = train[:,-1]
    
    onearr = map(lambda x: int(x) if x<9 else 9,X[:,-1]*10)
    twoarr = np.matrix(X[:,0]*X[:,-1])
    print np.shape(twoarr)
    bufarr = map(mask, onearr)
    #bufarr = np.zeros((len(y),10))
    X = np.append(X, bufarr,axis=1)
    #X = np.append(X, np.transpose(twoarr),axis=1)
    print np.shape(X)
    return X,y


In [76]:
X,y = gen("label2.tsv")


#print clf.coefs_paths_


(1, 393643)
(393643, 64)


In [77]:
print X
print (1-y)*99+1
clf = linear_model.LogisticRegressionCV()
clf.fit(X, y,sample_weight=(1-y)*99+1)
#z = clf.predict(X)
w = clf.predict_proba(X)

[[ 1.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  1.  0. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 1.  0.  1. ...,  0.  0.  1.]
 [ 0.  0.  1. ...,  0.  0.  1.]]
[   1.    1.    1. ...,  100.  100.  100.]


In [78]:
print clf.get_params()
print clf.coef_
print clf.intercept_

{'scoring': None, 'n_jobs': 1, 'verbose': 0, 'intercept_scaling': 1.0, 'fit_intercept': True, 'max_iter': 100, 'refit': True, 'penalty': 'l2', 'multi_class': 'ovr', 'random_state': None, 'dual': False, 'tol': 0.0001, 'Cs': 10, 'solver': 'lbfgs', 'cv': None, 'class_weight': None}
[[ -3.42784235e-02  -5.27665952e-02   1.98694110e-01  -8.43932555e-02
   -1.03508190e-01  -4.47743469e-02  -1.29261183e-01  -7.43929365e-02
   -1.94210170e-02  -6.19902131e-02  -8.89319974e-02  -1.64171793e-01
   -1.16659568e-01  -2.17613081e-01   1.98369145e-02  -4.15214316e-03
   -3.30413604e-02   1.72704347e-03  -3.36382053e-05  -1.34282153e-02
   -1.65339312e-03  -4.34332576e-03   1.44910722e-02  -1.07737868e-02
    2.54585724e-02   1.04596195e-02   1.22752961e-02   3.24817492e-03
   -2.82824724e-02  -2.49752334e-02   4.81968471e-03   1.32208998e-02
    4.79409049e-02  -2.56979125e-02  -1.42527965e-02   1.07583353e-03
   -3.17222325e-02  -3.97400879e-03   6.98062192e-03   1.34514608e-03
    2.67731553e-02  

In [79]:
click = y
#pctr = X[:,-1]
pctr = w[:,1]
#print click
#print display
print pctr
print metrics.roc_auc_score(click, pctr)
print metrics.roc_auc_score(click, X[:,53])
print X[:,53]
print X[:,2]
print metrics.roc_auc_score(click, X[:,53]*(X[:,2]* (0.2) + 0.9))


[ 0.00689125  0.03938439  0.00938145 ...,  0.01078253  0.01216081
  0.01354005]
0.673666595785
0.649310806019
[ 0.01  4.64  1.03 ...,  0.77  1.18  1.92]
[ 1.  0.  0. ...,  0.  1.  1.]
0.650996827842


In [37]:
XX,yy = gen("labeltest1.tsv")
ww = clf.predict_proba(XX)

#
#for xx,yy,ww in zip(X,y,w):
#    print xx[-1], yy, ww[1]

(100000, 54)


In [38]:
tempx = XX[2]
print tempx
print clf.predict_proba([tempx])
import math

print 1-1/(1+math.exp((tempx.dot(clf.coef_[0]))+clf.intercept_[0]))

[ 0.    0.    1.    0.25  0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    1.    0.
  0.    0.54]
[[ 0.98193211  0.01806789]]
0.018067889354


In [39]:
print metrics.roc_auc_score(yy, XX[:,-11])
print metrics.roc_auc_score(yy, XX[:,-11]*(XX[:,2]*(0.2)+0.9))
print metrics.roc_auc_score(yy, XX[:,-11]*(XX[:,2]*(0.2)+0.9)*(XX[:,-16]*(0.8)+XX[:,-15]*0.8+XX[:,-14]*1.2+XX[:,-13]*1.2+XX[:,-12]*1.2))
print metrics.roc_auc_score(yy, XX[:,-11]*(XX[:,-16]*(0.8)+XX[:,-15]*0.8+XX[:,-14]*1.2+XX[:,-13]*1.2+XX[:,-12]*1.2))
print metrics.roc_auc_score(yy, ww[:,1])
print metrics.roc_auc_score(yy, [1]*len(yy))

0.498882137927
0.498883263525
0.5
0.5
0.68023956225
0.5


In [28]:
print np.mean(y)
print np.std(y)
print np.mean(w[:,1])
print np.std(w[:,1])

0.496881692295
0.499990276063
0.0139055683892
0.0255514322491


In [41]:
print np.mean(yy)
print np.std(yy)
print np.mean(ww[:,1])
print np.std(ww[:,1])

0.00987
0.0988563761221
0.00971959086806
0.0103951711618


In [30]:
print np.mean(y)
print np.std(y)
print np.mean(w[:,1])
print np.std(w[:,1])
print np.mean(X[:,-11]/100)
print np.std(X[:,-11]/100)
print np.mean(XX[:,-11]/100)
print np.std(XX[:,-11]/100)

0.496881692295
0.499990276063
0.0139055683892
0.0255514322491
0.0134689586758
0.0175276254224
0.009538147
0.00950153566043
